# Volc_DEM
9/1/2022
This notebook reads in DEM data for the cascade volcanoes, visualizes the data, and stores it for later use

Import Modules

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import richdem as rd
from mpl_toolkits import mplot3d
from geopy import distance

In [ ]:
# define constants
pi = np.pi
color_scheme = 'jet'
radius = 6371e3

In [ ]:
# define dict with volc-center lat,lon,elev, and lower-left dem data corner lat,lon
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5, 46.625, -122.000,[]]
volc_lat_lon['Mt_Adams'] = [46.202621, -121.4906384, 3743.2, 46.125, -121.625,[]]
volc_lat_lon['Mt_Baker'] = [48.7773426,  -121.8132008, 3287.6, 48.750, -121.875,[]]
volc_lat_lon['Mt_St_Helens'] =[46.200472222222224,-122.18883611111112,2549, 46.125, -122.250,[]] 
volc_lat_lon['Glacier_Peak'] = [48.1112273, -121.1139922, 3213, 48, -121.25,[]]
volc_lat_lon['Crater_Lake']=[42.907745, -122.143494, 1883, 42, -123,[]]
volc_lat_lon['Mt_Hood']=[45.373221, -121.696509, 3428.7, 45, -122,[]]
volc_lat_lon['Newberry']=[43.7220653, -121.2344654, 2435, 43, -122,[]]

In [ ]:
volc_grid = {'Mt_Rainier': [46.74032510548371, -121.92393623921424, 24999.999843],
 'Mt_Adams': [46.106704928338665, -121.62841697638393, 21304.49616672],
 'Mt_Baker': [48.69314176638344, -121.94028162361221, 18711.439836960002],
 'Mt_St_Helens': [46.13517461639252, -122.28270726876059, 14507.683460400001],
 'Glacier_Peak': [48.02893063274638, -121.23658516139658, 18286.8524748],
 'Crater_Lake': [42.859490162690115, -122.20906103189014, 10717.131406800001],
 'Mt_Hood': [45.28535932500786, -121.82087738520083, 19514.51325252],
 'Newberry': [43.659665418808615, -121.32037100893913, 13858.850226]}


In [ ]:
# define function to get the distance of the lower-left corner to the center of the volcano
def dist(slat,slon,elat,elon,radius):
    lam = slon-elon
    g = (np.sin(slat)*np.sin(elat))+(np.cos(slat)*np.cos(elat)*np.cos(lam))
    d = radius*np.arccos(g)
    dist = d/10
    return dist

def start_latlon(slat, slon, x, y):
    x = x*10
    y = y*10
    hypotenuse = np.sqrt((x)**2+(y)**2)
    bearing = np.arctan(x/y)
    d = distance.geodesic(meters = hypotenuse)
    slat2 = d.destination(point=[slat,slon], bearing=bearing)[0]
    slon2 = d.destination(point=[slat,slon], bearing=bearing)[1]
    return slat2, slon2

In [ ]:
# merges different data files together for comprehensive data 
# only necessary for Washington Volcanoes
# for name in volc_lat_lon:
name = 'Glacier_Peak'
if volc_lat_lon[name][0] > 46:
    path = Path('DEM_data/'+name)
    path1 = Path('DEM_data/'+name+'/.ipynb_checkpoints')
    #path.mkdir(parents=True, exist_ok=True)
    out_path = 'DEM_data/'+str(name)+'/'+str(name)+'.tif'
    volc_files = list(path.iterdir())
    to_mosiac = []
    for d in volc_files:
        if d != path1:
            raster = rio.open(d)
            to_mosiac.append(raster)
    mosaic, output = merge(to_mosiac)
    output_meta = raster.meta.copy()
    output_meta.update(
        {'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': output,})
    with rio.open(out_path, 'w', **output_meta) as m:
        m.write(mosaic)


In [ ]:
dem = rio.open('DEM_data/Mt_Rainier/Mt_Rainier.tif')
dem_array = dem.read(1).astype('float64')

In [ ]:
dem = rio.open('DEM_data/Mt_Rainier/Mt_Rainier.tif')
dem_array = dem.read(1).astype('float64')
fig,ax = plt.subplots(1,1,figsize=(7,7))
dem_array[dem_array == -32767] = np.nan
ax.imshow(dem_array,extent=dem.bounds,cmap='gray')

In [ ]:
dem_array

In [ ]:
# getting the average slope for each volcano
volc = rd.rdarray(dem_array, no_data=-9999)  #need to figure out the size of grid
#slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
#aspect = rd.TerrainAttribute(volc, attrib = 'aspect')
fig1  = rd.rdShow(volc, axes=True, cmap=color_scheme, figsize=(15,15))
#fig2 = rd.rdShow(slope, axes=False, cmap=color_scheme, figsize=(15,8))
#fig3= rd.rdShow(aspect,axes =  True, cmap = color_scheme, figsize = (15,15))
#np.mean(slope)

Plot the data using matplotlib

In [ ]:
x,y = 100,100
height = dem_array.shape[0]
length = dem_array.shape[1]
volc = rd.rdarray(dem_array[100:height-100,100:length-100], no_data=-9999)
height = volc.shape[0]
length = volc.shape[1]

for name in volc_lat_lon:
    # new lower-left indices based on trimming of volc
    slat = volc_lat_lon[name][3]
    slon = volc_lat_lon[name][4]
    start_lat, start_lon = start_latlon(slat,slon,x,y)
    # vertical distance from lower left corner to volc-center 
    slat = np.deg2rad(start_lat)#slat = np.deg2rad(volc_lat_lon[name][0])
    elat = np.deg2rad(volc_lat_lon[name][0])
    slon = np.deg2rad(volc_lat_lon[name][1])
    elon = np.deg2rad(volc_lat_lon[name][1])
    v_index = dist(slat, slon, elat, elon, radius)
    volc_lat_lon[name][5].append(v_index)
    # horizontal distance from lower left corner to volc-center
    slat = np.deg2rad(volc_lat_lon[name][0])
    elat = np.deg2rad(volc_lat_lon[name][0])
    slon = np.deg2rad(start_lon)
    elon = np.deg2rad(volc_lat_lon[name][1])
    h_index = dist(slat, slon, elat, elon, radius)
    volc_lat_lon[name][5].append(h_index)
    break
# slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
# fig2 = rd.rdShow(slope, axes=False, cmap=color_scheme, figsize=(15,8))
# fig3 = rd.rdShow(aspect, axes=False, cmap=color_scheme, figsize=(15,8))
# aspect = rd.TerrainAttribute(volc, attrib = 'aspect')

# indices of the center of the volcano
y_index = volc_lat_lon['Mt_Rainier'][5][0]
x_index = volc_lat_lon['Mt_Rainier'][5][1]

fig, ax = plt.subplots(1,2, figsize=(20,10))
b = ax[0].contourf(volc,200,cmap = 'bone_r')
ax[0].scatter(x_index, y_index,s=200,marker='*',c='r')
ax[0].invert_yaxis()
ax[1].scatter(volc_lat_lon['Mt_Rainier'][1],volc_lat_lon['Mt_Rainier'][0], s = 100, c = 'r', marker = '*')
# fig.colorbar(b)

# test lat/lon data for stas on Mt_Rainier
lats = [46.98832, 46.988131,46.899719,46.731319,46.859032,46.786442,46.929694,46.841649,46.90306,46.867092,
46.96633,46.941399,46.750599,46.750599,46.83564,46.81905,46.850849,46.613279]
lons = [-122.00541,-121.961063,-121.815331,-121.571171,-121.642592,-121.742149,-121.988976,-121.948912,-121.94439,
-121.953246,-121.9834,-121.670998,-121.809601,-121.809601,-121.732979,-121.842133,-121.792953,-121.404485]
stas = ['CARB','CRBN','OBSR','OPCH','PANH','PARA','PR04','PR05','RUSH','SIFT','VOIT','FMW','LO2','LON','RCM','RER',
'STAR','WPEQ']

# plotting stations over top of DEM 
for i in range(0,len(lats)):
    v = dist(np.deg2rad(start_lat), np.deg2rad(lons[i]), np.deg2rad(lats[i]), np.deg2rad(lons[i]), radius)
    print(v)
    h_dist = dist(np.deg2rad(lats[i]),np.deg2rad(start_lon),np.deg2rad(lats[i]), np.deg2rad(lons[i]), radius)
    v_dist = dem_array.shape[0] - v
    if lats[i] < start_lat:
        v_dist = dem_array.shape[0] + v
    if lons[i] < start_lon:
        h_dist = -1*h_dist
    ax[0].scatter(h_dist,v_dist,s=100)
    ax[0].text(h_dist,v_dist,stas[i])
    ax[1].scatter(lons[i],lats[i],s = 100, c = 'k')
    ax[1].text(lons[i],lats[i],stas[i])


In [ ]:
# trying a different plotting method, using imshow
fig, ax = plt.subplots(1,2, figsize=(20,10))
a = ax[0].imshow(volc,cmap = 'jet')
aspect = rd.TerrainAttribute(volc, attrib = 'aspect')
b = ax[1].imshow(aspect,cmap = 'jet')


In [ ]:
#units of Rainier are in meters
#units of St Helens are in feet
#units of Glacier in meters
#takes longer to plot with matplotlib 